In [4]:
import pandas as pd
import time
import sys
from pathlib import Path

# Go up one directory to get to master/
project_root = str(Path.cwd().parent)
sys.path.append(project_root)



In [5]:
max_depth_available = 8
name_dataset = "adult"

In [ ]:
def get_solutions_all_folds_pulp(name_dataset: str, depth=3) -> pd.DataFrame: #for 10 fold cross valdidation, carefull that trees have min depth 2
    sol_dict = {} # one entry for every fold, [fold][depth]['test'] for test_data classification for fold of depth; same for training
    for i in range(1,11):
        sol_dict[i] = {}
        for j in range(2,depth+1):
            sol_dict[i][j] = {}
            sol_dict[i][j]['test'] = pd.read_csv(f"../results/{name_dataset}/pulp/fold{i}/depth{j}_classification_{name_dataset}_test.csv")
            sol_dict[i][j]['train'] = pd.read_csv(f"../results/{name_dataset}/pulp/fold{i}/depth{j}_classification_{name_dataset}_train.csv")
    return sol_dict

sol_dict = get_solutions_all_folds_pulp(name_dataset=name_dataset, depth= max_depth_available)

#print(sol_dict[4][8]['test'])

      Unnamed: 0  y  prediction  leaf
0              3  1           1   447
1              4  1           2   278
2             15  1           1   433
3             26  1           1   471
4             40  1           1   439
...          ... ..         ...   ...
4879       48802  1           1   335
4880       48805  2           1   423
4881       48808  1           1   467
4882       48822  2           2   285
4883       48838  1           1   478

[4884 rows x 4 columns]


damit das hier immer funktioniert, muss das mapping vom binarizer auf die Zahlen immer gleich sein

Achtung: Man muss noch in den Ergebnissen zuordnen was in den Ursprungsdatensatz (das binarized wurde) als positive und was als negative gesehen wird. Also wie das label jetzt heißt dem ursprünglichen label zuordnen

=> Ich habe mich entschieden das schon beim binarizen zu machen

In [ ]:
# specifics for every dataset can of course vary

if name_dataset == "adult": #we are looking to match <=50 and >50 to binary labels
    # wie war das nochmal mit den Zuordnungen und warum mach ich hier nicht die Zuordnung über dem binarized dataset vor rollOCT
    origin_dataset_test = pd.read_csv(f"../datasets/{name_dataset}/adult.test", sep=',', skipinitialspace=True, header=None) #16281 rows
    # Remove dots from the 'target' column
    origin_target_col = origin_dataset_test[14].astype(str).str.replace('.', '', regex=False) #in this dataset targets are in col 14
    some_dataset_solution = sol_dict[1][max_depth_available]['test']
    target_labels_col = some_dataset_solution['y']
    df = pd.concat([origin_target_col, target_labels_col], axis=1)
    print(df)
    mapping = df.drop_duplicates().set_index('y')[14].to_dict() #this doesnt fail if number of rows dont match exactly. Missing col entries get NaN
    #print(mapping)
    #Convention for now: >50K is positive, <=50K is negative

          14    y
0      <=50K  1.0
1      <=50K  1.0
2       >50K  1.0
3       >50K  1.0
4      <=50K  1.0
...      ...  ...
16276  <=50K  NaN
16277  <=50K  NaN
16278  <=50K  NaN
16279  <=50K  NaN
16280   >50K  NaN

[16281 rows x 2 columns]


In [49]:
print(target_labels_col)
print(prediction_col)

0        1
1        1
2        2
3        2
4        1
        ..
16276    1
16277    1
16278    1
16279    1
16280    2
Name: y, Length: 16281, dtype: int64
0        1
1        1
2        1
3        2
4        1
        ..
16276    1
16277    1
16278    2
16279    1
16280    2
Name: prediction, Length: 16281, dtype: int64


In [ ]:
def acc_sens_spec_prec(solution, pos_represent, neg_represent): #16281
    tp = len(solution[(solution['y'] == pos_represent) & (solution['prediction'] == pos_represent)])
    tn = len(solution[(solution['y'] == neg_represent) & (solution['prediction'] == neg_represent)])
    fp = len(solution[(solution['y'] == neg_represent) & (solution['prediction'] == pos_represent)])
    fn = len(solution[(solution['y'] == pos_represent) & (solution['prediction'] == neg_represent)])
    number_datapoints = tp+tn+fp+fn
    if number_datapoints != len(solution):
        print('error in calculation of acc')
    sensitivity = tp/(tp+fn) #also called recall, True Positive Rate
    specificity = tn/(tn+fp)
    precision = tp/(tp+fp)
    accuracy = (tp+tn)/number_datapoints
    return accuracy, sensitivity, specificity,precision




acc, sens, spec, prec = acc_sens_spec_prec(solution=solution, pos_represent=2, neg_represent=1)
print(acc_sens_spec_prec(solution=solution, pos_represent=2, neg_represent=1))

16281
16281
